# Homework 5

In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split


## Download Data

In [10]:
df = pd.read_csv('train.csv')
df.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_test = pd.read_csv('test.csv')
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preparation

### Exporting first 10 rows as images

In [2]:
i = 0
with open('train.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header row if exists
    
    for data in csv_reader:
        # First column is label (0-9)
        label = int(data[0])
        
        # Remaining columns are pixels (0-255)
        pixels = data[1:]  # Skip first column
        pixels = np.array(pixels, dtype='uint8')
        
        # Reshape to 28x28
        pixels = pixels.reshape((28, 28))
        
        # Save image
        plt.imshow(pixels, cmap='gray')
        plt.title(f'Label: {label}')
        plt.savefig(f'image_{i}_label_{label}.png')
        plt.close()
        
        i += 1
        
        if i >= 10:  # Limit to first 10 images for testing
            break

### See if there are any missing values

In [12]:
print("\nMissing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Missing Values:
Series([], dtype: int64)


### Split data into $X$ and $y$

In [13]:
X = df.drop('label', axis=1)
y = df['label']

### Normalize pixel values

In [14]:
# for training set
X = X / 255

#for test set
df_test = df_test / 255


## Build and train model

### Train test split

In [15]:
# Split data for a train and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Fitting

In [7]:
# initialize the model
mlp = MLPClassifier(activation='logistic') 

# fit the model
mlp.fit(X_train, y_train)

# make predictions
y_pred = mlp.predict(X_test)

# calculate accuracy
mlp.score(X_test, y_test)

0.9704761904761905

### Predicting

In [16]:
predictions = mlp.predict(df_test)

predictions_df = pd.DataFrame({
    'ImageId': df_test.index + 1,  # use row indexes and start from 1 (since it is like this in the sample submission)
    'Label': predictions  # Predictions from the model
})


predictions_df.to_csv('first_try_no_tuning.csv', index=False)


